In [ ]:
import langchain_core
dir(langchain_core)

['VERSION',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_api',
 '_import_utils',
 'caches',
 'callbacks',
 'documents',
 'embeddings',
 'env',
 'exceptions',
 'globals',
 'language_models',
 'load',
 'messages',
 'output_parsers',
 'outputs',
 'prompt_values',
 'rate_limiters',
 'runnables',
 'surface_langchain_beta_warnings',
 'surface_langchain_deprecation_warnings',
 'tools',
 'tracers',
 'utils',
 'version']

In [132]:
!pip install -q -U langchain langchain-core langchain-community langchain-openai langchain-huggingface

In [154]:
!pip install -q pypdf

In [133]:
from langchain_core.prompts import PromptTemplate

In [134]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

In [135]:
from langchain_community.document_loaders import PyPDFLoader

In [136]:
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough

In [137]:
from langchain_openai import OpenAI

In [138]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

In [139]:
def Get_pdf_text(path:dict)->str:
  if path!=None:
    pdf_url=path['input']
  else:
    pdf_url='/content/ukpga_20250022_en.pdf'
  pdf_loader=PyPDFLoader(pdf_url)
  pages=pdf_loader.load()
  full_text=''
  for p in pages:
    full_text+=p.page_content
  return full_text

In [140]:
Get_text=RunnableLambda(Get_pdf_text)

In [141]:
from google.colab import userdata
api_key=userdata.get('PROJECT')
llm=HuggingFaceEndpoint(
    repo_id='openai/gpt-oss-20b',
    huggingfacehub_api_token=api_key,
    temperature=0.2
        )

In [142]:
llm_chat=ChatHuggingFace(llm=llm)

In [144]:
summary_template='''
Your a halpful assistance.
Task:Summarize the given Act context.

Instructions:
1.Focus on the following points in the summary:
- Purpose
- Key definitions
- Eligibility
- Obligations
- Enforcement elements
2. Pay attention that the summary should not be less than 5 bullet points and not exceed more than 10 bullet points.

Fromat:Json.
Example:{{
  'summary_points':'['1.Point1', '2.Point2'.......'Point10']'}}

Context:
{context}
'''

In [145]:
summary_prompt=PromptTemplate(template=summary_template)

In [146]:
summary_chain=(
    {"context": lambda x: x}
    | summary_prompt
    | llm_chat
    | JsonOutputParser()
)

In [147]:
KeyExtractions_template='''
Your a helpful assitant.
Task: Extract key Legislative Sections from the given Act context.

Instructions:
1. The following Key should be extracted:
- Definitions
- Obligations
- Responsibilities
- Eligibility
- Payments / Entitlements
- Penalties / Enforcement
- Record-keeping / Reporting

2. Pay attention the result should be strictly in the Json format.

Fromat: Json with the following keys:
-definations
-obligations
-responsibilities
-eligibility
-payments
-panalties
-record_keeping

Context:
{context}
'''

In [148]:
KeyExtractions_prompt=PromptTemplate(template=KeyExtractions_template)

In [149]:
KeyExtraction_chain=(
    {'context': lambda x:x}
    | KeyExtractions_prompt
    | llm_chat
    | JsonOutputParser()
)

In [150]:
RuleCheck_template='''
Your a smart and helpful assistant.
Task:Appy the following rule checks on the given Act context.
Instructions:
1. The following rules should be checked:
-Act must define key terms
-Act must specify eligibility criteria
-Act must specify responsibilities of the administering authority
-Act must include enforcement or penalties
-Act must include payment calculation or entitlement structure
-Act must include record-keeping or reporting requirements

2. Pay attention the result should be strictly in the Json format.

Fromat: For each rule check the Json should consist the following keys:
-rule
-status
-evidence
-confidence

Example for one such rule Check is:
{{
 "rule": "Act must define key terms",
 "status": "pass",
 "evidence": "Section 2 – Definitions",
 "confidence": 92

}}

Act Context:
{Context}
'''

In [151]:
RuleCheck_prompt=PromptTemplate(template=RuleCheck_template)

In [152]:
RuleCheck_chain=(
    {'Context': lambda x:x}
    | RuleCheck_prompt
    | llm_chat
    | JsonOutputParser()
)

In [153]:
parallel_chains=RunnableParallel(
    summary=summary_chain,
    key_extraction=KeyExtraction_chain,
    rule_check=RuleCheck_chain
)

In [155]:
Full_chain=(
    Get_text
    | parallel_chains
)

In [156]:
Full_chain.invoke({'input':'/content/ukpga_20250022_en.pdf'})

{'summary': {'summary_points': ['1.Purpose: To adjust the rates of the standard allowance, limited capability for work and work-related activity (LCWRA) elements, and income-related ESA components for Universal Credit across 2026‑27 to 2029‑30, ensuring annual CPI and uplift adjustments are applied without automatic annual up‑rating.',
   "2.Key definitions: 'Standard allowance' – basic Universal Credit amount; 'LCWRA element' – additional amount for claimants with limited capability for work and work-related activity; 'protected LCWRA amount' – the fixed LCWRA for pre‑2026, severe conditions, or terminally ill claimants; 'cpi percentage' – year‑on‑year consumer price index change.",
   '3.Eligibility: Claimants must qualify under the usual Universal Credit criteria and, for LCWRA, must be pre‑2026, a severe conditions criteria claimant, or terminally ill; ESA legacy payments are adjusted for claimants with disabilities or work‑related components.',
   '4.Obligations: The Secretary of 